In [2]:
embeding_matrix = load_pickle_file("./embeding_matrix")
word2index = load_pickle_file("./word2index")
index_sequences = load_pickle_file("./index_sequences.pkl")
labels = load_pickle_file("./labels")
SEQUENCE_LENGTH = 618
VOCAB_SIZE = len(embeding_matrix)
EMB_DIMENSION = 300
VALIDATION_SPLIT = 0.1
PAD_SEQ_VALUE = 3001066

In [12]:
VOCAB_SIZE = len(embeding_matrix)
PAD_SEQ_VALUE = 3001066
labels = load_pickle_file("./labels")


In [3]:
import numpy as np
same_lenght_seq = [seq[:SEQUENCE_LENGTH].reshape(1,SEQUENCE_LENGTH) if len(seq) >= SEQUENCE_LENGTH 
                   else np.concatenate((seq,np.array([PAD_SEQ_VALUE] * (SEQUENCE_LENGTH - len(seq)))),axis = 0).reshape(1,SEQUENCE_LENGTH) 
                   for seq in index_sequences]
same_lenght_seq = np.concatenate(same_lenght_seq,axis=0)


In [13]:
import random
test_size = int(len(labels) * 0.2)
true_labels = np.where(np.array(labels)==0)[0]
start_index = true_labels[0]
random_test_true_indexes = [int(x * len(true_labels)+start_index) for x in np.random.random(test_size//2)]
random_test_false_indexes = [int(x * (len(labels) - len(true_labels))) for x in np.random.random(test_size//2)]
x_test_indexes = random_test_true_indexes + random_test_false_indexes
random.shuffle(x_test_indexes)

x_train_indexes = [x for x in range(0,len(labels)) if x not in x_test_indexes]

random.shuffle(x_train_indexes)

labels = np.vstack(np.array([1,0]).reshape(1,2) if x == 0 else np.array([0,1]).reshape(1,2) for x in labels)

y_test = np.array(labels)[x_test_indexes]
y_train = np.array(labels)[x_train_indexes]

x_train = same_lenght_seq[x_train_indexes]
x_test = same_lenght_seq[x_test_indexes]

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:14: FutureWarning: arrays to stack must be passed as a "sequence" type such as list or tuple. Support for non-sequence iterables such as generators is deprecated as of NumPy 1.16 and will raise an error in the future.
  


Dataset split

In [5]:
# from sklearn.model_selection import train_test_split

# x_train, x_test, y_train, y_test = train_test_split(same_lenght_seq, labels, test_size=0.2, random_state=42,shuffle = True)

In [14]:
data_size = len(x_train)
perm = np.random.permutation(data_size)
idx_train = perm[:int(data_size*(1-VALIDATION_SPLIT))]
idx_val = perm[int(data_size*(1-VALIDATION_SPLIT)):]

data_train = x_train[idx_train]
labels_train = y_train[idx_train]

data_val = x_train[idx_val]
labels_val = y_train[idx_val]

In [13]:
data_train.shape

(7380, 434)

<h1>LSTM</h1>

In [6]:
import tensorflow as tf
from tensorflow import keras as tk
from tensorflow.keras import layers as tfkl
from sklearn.metrics import mean_squared_error,mean_absolute_error

from tensorflow.keras.layers import Dense, Input, LSTM, Embedding, Dropout, Activation
from tensorflow.keras.models import Model
# from tensorflow.keras.layers.normalization import BatchNormalization
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint

/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/dtypes.py:523: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/dtypes.py:524: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/dtypes.py:525: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/dtypes.py:526: FutureWarning: Passing (type, 1) or 

In [8]:
model_checkpoint = ModelCheckpoint("./Checkpoints", save_best_only=True, save_weights_only=True)


In [26]:
LSTM_HIDDEN_DIM_SIZE = 64
rate_drop_lstm = 0.15
rate_drop_lstm = 0.15
lstm_model = tk.Sequential()
embedding_layer = Embedding(VOCAB_SIZE,
        EMB_DIMENSION,
        weights=[embeding_matrix],
        input_length=SEQUENCE_LENGTH,
        trainable=False)
lstm_model.add(embedding_layer)
lstm_model.add(LSTM(LSTM_HIDDEN_DIM_SIZE, activation = 'tanh', dropout=rate_drop_lstm, recurrent_dropout=rate_drop_lstm,return_sequences = True))
lstm_model.add(LSTM(LSTM_HIDDEN_DIM_SIZE//2, activation = 'tanh', dropout=rate_drop_lstm, recurrent_dropout=rate_drop_lstm))
lstm_model.add(Dense(LSTM_HIDDEN_DIM_SIZE//4, activation = 'relu'))
# lstm_model.add(LSTM(LSTM_HIDDEN_DIM_SIZE//4, activation = 'tanh',dropout=rate_drop_lstm, recurrent_dropout=rate_drop_lstm))
lstm_model.add(Dense(2,activation='sigmoid'))

In [25]:
del lstm_model

In [18]:
lstm_model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, 618, 300)          900320100 
_________________________________________________________________
unified_lstm_2 (UnifiedLSTM) (None, 618, 64)           93440     
_________________________________________________________________
unified_lstm_3 (UnifiedLSTM) (None, 32)                12416     
_________________________________________________________________
dense_2 (Dense)              (None, 16)                528       
_________________________________________________________________
dense_3 (Dense)              (None, 2)                 34        
Total params: 900,426,518
Trainable params: 106,418
Non-trainable params: 900,320,100
_________________________________________________________________


In [24]:
lstm_model.reset_states()

In [28]:
opt = tk.optimizers.Adam(learning_rate=0.007)
# opt = tk.optimizers.RMSprop(
#     learning_rate=0.002,
#     rho=0.9,
#     momentum=0.0,
#     epsilon=1e-07,
#     centered=False,
#     name="RMSprop"
# )

In [29]:
lstm_model.compile(loss='binary_crossentropy',
        optimizer=opt,
        metrics=['acc'])
lstm_model.fit(data_train,labels_train,epochs = 25,batch_size = 128, validation_data = [data_val,labels_val])

Train on 11700 samples, validate on 1300 samples
Epoch 1/25
11700/11700 [==============================] - 457s 39ms/sample - loss: 0.5738 - acc: 0.7323 - val_loss: 0.5760 - val_acc: 0.7273
Epoch 2/25
11700/11700 [==============================] - 428s 37ms/sample - loss: 0.5839 - acc: 0.7305 - val_loss: 0.5925 - val_acc: 0.7123
Epoch 3/25
11700/11700 [==============================] - 427s 37ms/sample - loss: 0.5669 - acc: 0.7312 - val_loss: 0.5922 - val_acc: 0.7127
Epoch 4/25
11700/11700 [==============================] - 440s 38ms/sample - loss: 0.5701 - acc: 0.7324 - val_loss: 0.5934 - val_acc: 0.7115
Epoch 5/25
11700/11700 [==============================] - 430s 37ms/sample - loss: 0.5627 - acc: 0.7313 - val_loss: 0.5606 - val_acc: 0.7123
Epoch 6/25
11700/11700 [==============================] - 430s 37ms/sample - loss: 0.5790 - acc: 0.7314 - val_loss: 0.6022 - val_acc: 0.7123
Epoch 7/25
11700/11700 [==============================] - 444s 38ms/sample - loss: 0.5791 - acc: 0.7314 -

In [31]:
lstm_model.fit(data_train,labels_train,epochs = 5,batch_size = 128, validation_data = [data_val,labels_val])

Train on 11700 samples, validate on 1300 samples
Epoch 1/5
11700/11700 [==============================] - 436s 37ms/sample - loss: 0.1208 - acc: 0.9566 - val_loss: 0.1914 - val_acc: 0.9288
Epoch 2/5
11700/11700 [==============================] - 436s 37ms/sample - loss: 0.1076 - acc: 0.9608 - val_loss: 0.1875 - val_acc: 0.9377
Epoch 3/5
11700/11700 [==============================] - 432s 37ms/sample - loss: 0.1034 - acc: 0.9608 - val_loss: 0.1884 - val_acc: 0.9377
Epoch 4/5
11700/11700 [==============================] - 436s 37ms/sample - loss: 0.0963 - acc: 0.9661 - val_loss: 0.1859 - val_acc: 0.9354
Epoch 5/5
11700/11700 [==============================] - 434s 37ms/sample - loss: 0.1120 - acc: 0.9588 - val_loss: 0.2166 - val_acc: 0.9315


In [29]:
lstm_model.save("./Checkpoints/lstm_model.h5")

In [30]:
lstm_model.save_weights("./Checkpoints/lstm_weights")

In [9]:
lstm_model = tf.keras.models.load_model("./Checkpoints/lstm_model.h5")

Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.


In [32]:
from sklearn.metrics import accuracy_score
from sklearn.metrics import f1_score
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score

pred = lstm_model.predict(x_test)
pred = [np.argmax(x) for x in pred]

print(f"F1-score : {f1_score([np.argmax(x) for x in y_test],pred,pos_label=1)}")
print(f"Accuracy-score : {accuracy_score([np.argmax(x) for x in y_test],pred)}")

print(f"Precision : {precision_score([np.argmax(x) for x in y_test],pred,pos_label = 1)}")
print(f"Recall  : {recall_score([np.argmax(x) for x in y_test],pred,pos_label = 1)}")

F1-score : 0.9082462253193961
Accuracy-score : 0.9001895135818067
Precision : 0.8404083825900054
Recall  : 0.9879974731522426


In [35]:
from sklearn.metrics import accuracy_score

pred = lstm_model.predict(x_test)
pred = [np.argmax(x) for x in pred]



In [36]:
from sklearn.metrics import f1_score
f1_score([np.argmax(x) for x in y_test],pred,pos_label=1)

0.9669737137721861

In [37]:
accuracy_score([np.argmax(x) for x in y_test],pred)

0.9535838332807073

In [44]:
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score


print(f"Precision : {precision_score([np.argmax(x) for x in y_test],pred,pos_label = 1)}")
print(f"Recall  : {recall_score([np.argmax(x) for x in y_test],pred,pos_label = 1)}")

0.9598572702943801

In [19]:
i=0
for seq in data_train[128:250]:
    for el in seq:
        if np.isnan(el):
            print(i)
                
#         if type(arr) is np.ndarray:
#             print(i)
    i+=1

In [1]:
import pickle
def save_pickle_file(file,file_path,protocol=None):
    with open(file_path,"wb") as f:
        if protocol:
            pickle.dump(file,f,protocol = protocol)
        else:
            pickle.dump(file,f)

def load_pickle_file(file_path, protocol=None):
     with open(file_path,"rb") as f:
        if protocol:
            return pickle.load(f)
        else:
            return pickle.load(f)
